In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "20"
from collections import defaultdict
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
folder = "../data/cna/genosurf_download/files/"

In [4]:
files = os.listdir(folder)
files = filter(lambda x:x.endswith(".bed"), files)
files = list(files)
s = set()
d = defaultdict(int)

d_new = defaultdict(int)



check = True

max_ = 0
min_ = 0
min_pos = 10000000
max_neg = -10000000

counts = defaultdict(int)

for file_name in tqdm(files):
    file_name = folder + file_name
    if check:
        pre_chr = None
    with open(file_name) as file:
#         print(file_name)
        file = map(lambda x:x.strip().split("\t"), file)
        file = map(lambda x:(x[0],int(x[1]), int(x[2]),x[3],int(float(x[4])),float(x[5]),x[6]), file)
        file = filter(lambda x:x[6] == 'Y' , file)
        
        if check:
            file = sorted(file)
                
        for (chrom, start,stop,strand,count,value,nocnv) in file:
#             print(file_name,(chrom, start,stop,strand,count,value,nocnv))
            if check: # check is there any overlap !!!!
                if pre_chr != chrom:
                    pre_chr = chrom
                else:
                    if start < pre_stop:
                        print(pre_stop,"********************************************************************************")
                        print(chrom, start,stop,strand,count,value,nocnv)

                pre_stop = stop
            
            s.add((chrom,start))
            s.add((chrom,stop))
            d[(chrom, start,stop)] = d[(chrom, start,stop)] + 1
            
            d_new[(chrom,start)] = d_new[(chrom,start)] + 1
            d_new[(chrom,stop)] = d_new[(chrom,stop)] - 1
            
            max_ = max(max_, value)

            min_ = min(min_, value)
            if value > 0:
                min_pos = min(min_pos, value )
            if value < 0:
                max_neg = max(max_neg, value)
                
            if value == 0:
                counts['value == 0'] = counts['value == 0']  + 1
            
            if value < 0:
                counts['value < 0'] = counts['value < 0']  + 1
                
            if value > 0:
                counts['value > 0'] = counts['value > 0']  + 1
            
            if value > 1:
                counts['value > 1'] = counts['value > 1']  + 1
            if value < -1:
                counts['value < -1'] = counts['value < -1']  + 1
                
            if value > 2:
                counts['value > 2'] = counts['value > 2']  + 1
            if value < -2:
                counts['value < -2'] = counts['value < -2']  + 1
            
            counts['all'] = counts['all']  + 1

        


100%|██████████| 22632/22632 [01:02<00:00, 361.63it/s]


In [140]:
b = map(lambda x:x*3, a)

In [141]:
sum(b)

3675

In [127]:
a = 0
np.sign(a)

0

In [15]:
for el in list(range(0,23))+list("X"): print("chr"+str(el))

chr0
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX


In [16]:
chromosomes = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

for abc in chromosomes: print(abc)

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX


In [92]:
d_new

defaultdict(int,
            {('chr15', 23687685): 22631,
             ('chr15', 41937460): -4,
             ('chr15', 41950155): 4,
             ('chr15', 41991076): 0,
             ('chr19', 10177629): -4,
             ('chr19', 10656442): -3,
             ('chr19', 10656804): 4,
             ('chr19', 34599808): 12,
             ('chr19', 34600936): 0,
             ('chr19', 34661697): -1,
             ('chr19', 34667849): 2,
             ('chr19', 35075455): -4,
             ('chr19', 35084293): 8,
             ('chr19', 35154447): -8,
             ('chr19', 35156245): 4,
             ('chr19', 52270087): -38,
             ('chr19', 52624873): 35,
             ('chr19', 52653973): 0,
             ('chr15', 41992272): -9,
             ('chr15', 101884307): -22630,
             ('chr19', 52657023): 2,
             ('chr19', 58878226): -22632,
             ('chr16', 653459): 22552,
             ('chr16', 901174): -2,
             ('chr20', 455764): 22552,
             ('chr20', 207554

In [ ]:
chromosomes = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

In [4]:
np.unique(list(map(lambda x : x[0], d_new.keys())))

array(['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21',
       'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9',
       'chrX'], dtype='<U5')

In [5]:
# in order to use the positions, it is enough sets in the previous cell!!!!

chr_positions =sorted(map(lambda x : x[1], filter(lambda x : x[0] == 'chr1', d_new.keys())))
df = pd.DataFrame(index = chr_positions)
indexes = df.index
print('len(chr11_positions)',len(chr_positions))
all_series = []
for file_name in tqdm(files):
    new_series = pd.Series(np.nan, index=indexes, name=file_name)

    file_name = folder + file_name
    if check:
        pre_chr = None
    with open(file_name) as file:
#         print(file_name)
        file = map(lambda x:x.strip().split("\t"), file)
        file = map(lambda x:(x[0],int(x[1]), int(x[2]),x[3],int(float(x[4])),float(x[5]),x[6]), file)
        file = filter(lambda x:x[6] == 'Y' , file)

        file = sorted(file)

        for (chrom, start,stop,strand,count,value,nocnv) in file:
            if chrom != 'chr1':
                continue
            new_series[(start<=indexes) & (indexes<=stop)] = value
    all_series.append(new_series)


  0%|          | 3/22632 [00:00<17:45, 21.24it/s]

len(chr11_positions) 106960


100%|██████████| 22632/22632 [06:15<00:00, 60.30it/s]


In [6]:
all_series = pd.concat(all_series, axis=1).T

In [7]:
print(all_series.shape)
all_series.head()

(22632, 106960)


,3218610,3220101,3220160,3221051,3221621,3222113,3222334,3222470,3222627,3224658,...,247793202,247797469,247798522,247800377,247806068,247806930,247807327,247810288,247812431,247813706
TCGA-AA-3693-01A-01D-0903-01.bed,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,...,0.0057,0.0057,0.0057,0.0057,0.0057,0.0057,0.0057,0.0057,0.0057,0.0057
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,...,0.0221,0.0221,0.0221,0.0221,0.0221,0.0221,0.0221,0.0221,0.0221,0.0221
TCGA-24-0979-01A-01D-0428-01.bed,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,...,0.2419,0.2419,0.2419,0.2419,0.2419,0.2419,0.2419,0.2419,0.2419,0.2419
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,...,0.0272,0.0272,0.0272,0.0272,0.0272,0.0272,0.0272,0.0272,0.0272,0.0272
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,...,-0.0332,-0.0332,-0.0332,-0.0332,-0.0332,-0.0332,-0.0332,-0.0332,-0.0332,-0.0332


In [8]:
all_series.fillna(0, inplace=True)

In [8]:
all_series.columns[:-1]

Int64Index([  3218610,   3220101,   3220160,   3221051,   3221621,   3222113,
              3222334,   3222470,   3222627,   3224658,
            ...
            247792688, 247793202, 247797469, 247798522, 247800377, 247806068,
            247806930, 247807327, 247810288, 247812431],
           dtype='int64', length=106959)

In [9]:
all_series.columns[:-1]

Int64Index([  3218610,   3220101,   3220160,   3221051,   3221621,   3222113,
              3222334,   3222470,   3222627,   3224658,
            ...
            247792688, 247793202, 247797469, 247798522, 247800377, 247806068,
            247806930, 247807327, 247810288, 247812431],
           dtype='int64', length=106959)

In [12]:
corrs = []
has_nas = []
from scipy.stats import pearsonr
for x,y in tqdm(list(zip(all_series.columns[:-1],all_series.columns[1:]))):
    # in order understand the if the corrs change because of the nulls    
    has_na =  (sum(all_series[x].isna()) + sum(all_series[y].isna()) > 0)
    has_nas.append(has_na) 

    corr, z = pearsonr(all_series[x].fillna(0), all_series[y].fillna(0))
    #  print(x,y,corr, z)

    corrs.append(corr)


100%|██████████| 52847/52847 [06:37<00:00, 132.86it/s]


In [13]:
corrs[:10]

[1.0,
 0.9981826609331554,
 0.9998737381997217,
 1.0,
 0.9984791584968056,
 0.9994646039609068,
 0.9989558595339938,
 0.9987992490153113,
 0.9999351566475025,
 0.9994701132950817]

In [14]:
df = pd.DataFrame([corrs,has_nas]).T

In [15]:
df.shape

(52847, 2)

In [16]:
df.head(5)

,0,1
0,1,False
1,0.998183,False
2,0.999874,False
3,1,False
4,0.998479,False


In [17]:
corrs2 = df[df[1] == False][0]
corrs3 = df[df[1] == True][0]

In [18]:
print(corrs2.head(5))
corrs3.head(5)

0           1
1    0.998183
2    0.999874
3           1
4    0.998479
Name: 0, dtype: object


Series([], Name: 0, dtype: object)

In [19]:
from statistics import mean,median
mean(corrs),median(corrs),max(corrs),min(corrs)

(0.9830731075551162, 0.9980168258878328, 1.0, 0.39488908915125764)

In [20]:
mean(corrs2),median(corrs2),max(corrs2),min(corrs2)

(0.9830731075551162, 0.9980168258878328, 1.0, 0.39488908915125764)

In [21]:
from statistics import mean,median
mean(corrs3),median(corrs3),max(corrs3),min(corrs3)

StatisticsError: mean requires at least one data point

In [85]:
th = 0.6
print('all', len(df[(df[0] < th) ]) )
print('False', len(df[(df[0] < th) & (df[1] == False) ]) )
print('True', len(df[(df[0] < th) & (df[1] == True) ]) )

all 64
False 64
True 0


In [46]:
df.head()

,0,1
0,1,False
1,0.998183,False
2,0.999874,False
3,1,False
4,0.998479,False


In [24]:
df.loc[106958]

0    0.988294
1        True
Name: 106958, dtype: object

In [42]:
df.shape

(52847, 2)

In [28]:
all_series.columns[106807]

247567932

In [76]:
df.head()

,0,1
0,1,False
1,0.998183,False
2,0.999874,False
3,1,False
4,0.998479,False


In [47]:
df[(df[0] < th)].head()

,0,1
1490,0.662153,False
1492,0.657937,False
1509,0.625432,False
1511,0.640546,False
2132,0.67245,False


In [53]:
all_series.head()

,3157107,3157273,3163494,3165068,3165124,3166148,3167032,3167102,3169781,3170664,...,154883512,154883634,154884030,154884069,154884088,154895152,154895182,154895283,154896266,154905589
TCGA-AA-3693-01A-01D-0903-01.bed,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,0.0072,...,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004,-0.0004
TCGA-4P-AA8J-01A-11D-A390-01.bed,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,0.0192,...,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068,0.0068
TCGA-24-0979-01A-01D-0428-01.bed,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,0.1520,...,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317,0.6317
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,...,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017,0.0017
TCGA-5P-A9KC-01A-11D-A42I-01.bed,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,0.0047,...,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222,0.0222


In [81]:
init_idx = 0
new_df = pd.DataFrame()
for col_idx in tqdm(df[(df[0] < th)].index):
    aux = all_series.iloc[:,init_idx:col_idx].mean(axis=1)
        
    new_df["chrX_"+str(all_series.columns[init_idx] +np.sign(init_idx) )+":"+str(all_series.columns[col_idx])] = aux
    init_idx = col_idx
new_df["chrX_"+str(all_series.columns[init_idx])+":"+str(all_series.columns[-1])] = all_series.iloc[:,init_idx:].mean(axis=1)
new_df

100%|██████████| 221/221 [00:13<00:00, 16.63it/s]


,chrX_3157107:9886194,chrX_9886195:9888242,chrX_9888243:10433428,chrX_10433429:10434215,chrX_10434216:11721209,chrX_11721210:11722497,chrX_11722498:12175706,chrX_12175707:12175929,chrX_12175930:15727240,chrX_15727241:15731340,...,chrX_146053430:149512905,chrX_149512906:149516745,chrX_149516746:150940194,chrX_150940195:150943361,chrX_150943362:151149982,chrX_151149983:151150133,chrX_151150134:152705438,chrX_152705439:153762695,chrX_153762696:153765166,chrX_153765166:154905589
TCGA-AA-3693-01A-01D-0903-01.bed,0.007200,0.0072,0.0072,0.0072,0.007200,0.0072,0.0072,0.007200,0.007200,0.0072,...,-0.000400,-0.00040,-0.000400,-0.0004,-0.000400,-0.0004,-0.000400,-0.000400,-0.0004,-0.000400
TCGA-4P-AA8J-01A-11D-A390-01.bed,0.015987,0.0206,0.0206,0.0206,0.020600,0.0206,0.0206,0.020600,0.007772,0.0068,...,0.006800,0.00680,0.006800,0.0068,0.006800,0.0068,0.006800,0.006800,0.0068,0.006800
TCGA-24-0979-01A-01D-0428-01.bed,0.152000,0.1520,0.1520,0.1520,-0.682783,-0.7120,-0.7120,-0.712000,-0.712000,-0.7120,...,0.594100,0.59410,0.600773,0.6317,0.631700,0.6317,0.631700,0.631700,0.6317,0.631700
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.001700,0.0017,0.0017,0.0017,0.001700,0.0017,0.0017,0.001700,0.001700,0.0017,...,0.001700,0.00170,0.001700,0.0017,0.001700,0.0017,0.001700,0.001700,0.0017,0.001700
TCGA-5P-A9KC-01A-11D-A42I-01.bed,0.004700,0.0047,0.0047,0.0047,0.004700,0.0047,0.0047,0.004700,0.004700,0.0047,...,0.010300,0.01030,0.008407,0.0222,0.022200,0.0222,0.022200,0.022200,0.0222,0.022200
TCGA-19-0955-01A-02D-0517-01.bed,0.006100,0.0061,0.0061,0.0061,0.006100,0.0061,0.0061,0.006100,0.006100,0.0061,...,0.012100,0.01210,0.012100,0.0121,0.012100,0.0121,0.012100,0.012100,0.0121,0.012100
TCGA-FS-A4FD-10B-01D-A25P-01.bed,0.002200,0.0022,0.0022,-1.9539,0.004187,0.0105,0.0105,0.010500,0.010500,0.0105,...,0.010500,0.01050,0.010500,0.0105,0.010500,0.0105,0.010500,0.010500,0.0105,0.010500
TCGA-ZF-A9R5-10A-01D-A42G-01.bed,0.000200,0.0002,0.0002,0.0002,0.000200,0.0002,0.0002,0.000200,0.000200,0.0002,...,0.007300,0.00730,0.007300,0.0073,0.007300,0.0073,0.007300,0.007300,0.0073,0.007300
TCGA-D3-A51J-06A-11D-A25P-01.bed,0.051500,0.0515,0.0515,0.0515,0.051500,0.0515,0.0515,0.051500,0.051500,0.0515,...,0.058100,0.05810,0.058100,0.0581,0.058100,0.0581,0.058100,0.058100,0.0581,0.058100
TCGA-KN-8433-01A-11D-2308-01.bed,0.053300,0.0533,0.0533,0.0533,0.053300,0.0533,0.0533,0.053300,0.053300,0.0533,...,0.081700,0.08170,0.081700,0.0817,0.081700,0.0817,0.081700,0.081700,0.0817,0.081700


In [54]:
all_series.iloc[:,1490]

TCGA-AA-3693-01A-01D-0903-01.bed    0.0072
TCGA-4P-AA8J-01A-11D-A390-01.bed    0.0206
TCGA-24-0979-01A-01D-0428-01.bed    0.1520
TCGA-4W-AA9T-01A-11D-A390-01.bed    0.0017
TCGA-5P-A9KC-01A-11D-A42I-01.bed    0.0047
TCGA-19-0955-01A-02D-0517-01.bed    0.0061
TCGA-FS-A4FD-10B-01D-A25P-01.bed    0.0022
TCGA-ZF-A9R5-10A-01D-A42G-01.bed    0.0002
TCGA-D3-A51J-06A-11D-A25P-01.bed    0.0515
TCGA-KN-8433-01A-11D-2308-01.bed    0.0533
TCGA-44-2655-01A-01D-1549-01.bed    0.1073
TCGA-SR-A6MS-10A-01D-A35F-01.bed    0.0095
TCGA-EE-A29A-10A-01D-A195-01.bed    0.0139
TCGA-BR-4197-11A-01D-1130-01.bed    0.0013
TCGA-BH-A0BM-01A-11D-A059-01.bed   -0.0007
TCGA-KT-A7W1-01A-11D-A349-01.bed    0.0087
TCGA-B2-4098-11A-01D-1185-01.bed   -0.0112
TCGA-CR-7391-01A-11D-2010-01.bed    0.0016
TCGA-E2-A1B1-10A-01D-A12N-01.bed    0.0022
TCGA-A7-A426-10A-01D-A242-01.bed    0.0034
TCGA-FI-A3PX-10A-01D-A229-01.bed    0.0016
TCGA-22-5478-11A-11D-1631-01.bed    0.0071
TCGA-R6-A6L4-10A-01D-A31T-01.bed    0.0114
TCGA-NJ-A4Y

In [75]:
new_df.isnull().sum().sum()

0

In [49]:
new_df.shape

(22632, 222)

In [51]:
new_df

,chrX_3157107:9886194,chrX_9888179:9888242,chrX_9888383:10433428,chrX_10433550:10434215,chrX_10435519:11721209,chrX_11721459:11722497,chrX_11722528:12175706,chrX_12175845:12175929,chrX_12176019:15727240,chrX_15727977:15731340,...,chrX_146054140:149512905,chrX_149516457:149516745,chrX_149517397:150940194,chrX_150940266:150943361,chrX_150944418:151149982,chrX_151150032:151150133,chrX_151150476:152705438,chrX_152706248:153762695,chrX_153764217:153765166,chrX_153766111:154905589
TCGA-AA-3693-01A-01D-0903-01.bed,0.007200,0.0072,0.0072,0.0072,0.007200,0.0072,0.0072,0.0072,0.007200,0.0072,...,-0.000400,-0.0004,-0.000400,-0.0004,-0.000400,-0.0004,-0.000400,-0.000400,-0.0004,-0.00040
TCGA-4P-AA8J-01A-11D-A390-01.bed,0.015987,0.0206,0.0206,0.0206,0.020600,0.0206,0.0206,0.0206,0.007764,0.0068,...,0.006800,0.0068,0.006800,0.0068,0.006800,0.0068,0.006800,0.006800,0.0068,0.00680
TCGA-24-0979-01A-01D-0428-01.bed,0.152000,0.1520,0.1520,0.1520,-0.684129,-0.7120,-0.7120,-0.7120,-0.712000,-0.7120,...,0.594100,0.5941,0.600783,0.6317,0.631700,0.6317,0.631700,0.631700,0.6317,0.63170
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.001700,0.0017,0.0017,0.0017,0.001700,0.0017,0.0017,0.0017,0.001700,0.0017,...,0.001700,0.0017,0.001700,0.0017,0.001700,0.0017,0.001700,0.001700,0.0017,0.00170
TCGA-5P-A9KC-01A-11D-A42I-01.bed,0.004700,0.0047,0.0047,0.0047,0.004700,0.0047,0.0047,0.0047,0.004700,0.0047,...,0.010300,0.0103,0.008405,0.0222,0.022200,0.0222,0.022200,0.022200,0.0222,0.02220
TCGA-19-0955-01A-02D-0517-01.bed,0.006100,0.0061,0.0061,0.0061,0.006100,0.0061,0.0061,0.0061,0.006100,0.0061,...,0.012100,0.0121,0.012100,0.0121,0.012100,0.0121,0.012100,0.012100,0.0121,0.01210
TCGA-FS-A4FD-10B-01D-A25P-01.bed,0.002200,0.0022,0.0022,-3.9100,0.010500,0.0105,0.0105,0.0105,0.010500,0.0105,...,0.010500,0.0105,0.010500,0.0105,0.010500,0.0105,0.010500,0.010500,0.0105,0.01050
TCGA-ZF-A9R5-10A-01D-A42G-01.bed,0.000200,0.0002,0.0002,0.0002,0.000200,0.0002,0.0002,0.0002,0.000200,0.0002,...,0.007300,0.0073,0.007300,0.0073,0.007300,0.0073,0.007300,0.007300,0.0073,0.00730
TCGA-D3-A51J-06A-11D-A25P-01.bed,0.051500,0.0515,0.0515,0.0515,0.051500,0.0515,0.0515,0.0515,0.051500,0.0515,...,0.058100,0.0581,0.058100,0.0581,0.058100,0.0581,0.058100,0.058100,0.0581,0.05810
TCGA-KN-8433-01A-11D-2308-01.bed,0.053300,0.0533,0.0533,0.0533,0.053300,0.0533,0.0533,0.0533,0.053300,0.0533,...,0.081700,0.0817,0.081700,0.0817,0.081700,0.0817,0.081700,0.081700,0.0817,0.08170


In [160]:
chr_idx="X"

In [161]:
abc = 0.6 if chr_idx == "X" else 0.8
abc

0.6

In [22]:
len(s)/len(files)

57.99712796041004

In [23]:
len(d)

2584639

In [24]:
len(list(filter(lambda x: x[1]>1000, d.items())))

24

In [25]:
len(d_new)

1312591

In [26]:
sl = sorted(d_new.items())
sl[:10]

[(('chr1', 3218610), 22632),
 (('chr1', 3220101), -1),
 (('chr1', 3220160), 1),
 (('chr1', 3221051), -1),
 (('chr1', 3221621), -2),
 (('chr1', 3222113), 3),
 (('chr1', 3222334), -2),
 (('chr1', 3222470), -3),
 (('chr1', 3222627), 5),
 (('chr1', 3224658), -1)]

In [ ]:
new_list = []

chroms = sorted(set(map(lambda x:x[0],d_new.keys())))
for chrom in chroms:
    sl_chr = sorted(map(lambda x: (x[0][1],x[1]) ,filter(lambda x: x[0][0] == chrom, d_new.items())))
    pre, count = sl_chr[0]
    for curr, delta in sl_chr[1:]:
        if delta != 0:
            new_list.append((chrom, pre, curr, count))
            
        count += delta
        pre = curr


In [ ]:
len(new_list)

In [ ]:
new_list[:10]

In [ ]:
len(list(filter(lambda x:x[3] == 22632, new_list)))

In [ ]:
list(map(lambda x:x[3], new_list)))

In [ ]:
new_new_list = []
th = 0
total = 22632
c = 0
pre_chr, pre_start, pre_stop, pre_count = new_list[0]
for curr_chr, curr_start, curr_stop, curr_count in new_list:
    if curr_count + th < total:
        c+=1
        
        

c

In [ ]:
pre_stop = 0
cut = []
for curr_chr, curr_start, curr_stop, curr_count in new_list:
#     print("NEWLINE:", (curr_chr, curr_start, curr_stop, curr_count))
    if curr_start > pre_stop:
#         print("CUT:", pre_stop, curr_start, curr_start - pre_stop)
        cut.append((pre_stop, curr_start, curr_start - pre_stop))
        pre_start = curr_start
        
    pre_stop = curr_stop
    
        

In [ ]:
len(cut)

In [ ]:
cut[:10]

In [107]:
pd.read_pickle("../data/cna/tcga_cna_raw_all_samples_all_chr.pkl").head(5)

,chr1_0:4371,chr1_4372:4373,chr1_4374:9615,chr1_9616:9617,chr1_9618:12627,chr1_12628:12629,chr1_12630:18074,chr1_18075:18076,chr1_18077:18633,chr1_18634:18635,...,chr20_0:8345,chr20_8346:14270,chr20_14271:18077,chr20_18078:18080,chr21_0:164,chr21_165:5556,chr21_5557:5559,chr21_5560:15774,chr21_15775:15776,chr22_0:13709
TCGA-AA-3693-01A-01D-0903-01.bed,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.006600,-0.0066,-0.0066,-0.0066,...,-0.760800,-0.157315,0.940500,0.9405,-0.0193,-0.019300,-0.0193,-0.019300,-0.0193,0.045700
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.040900,-0.0409,-0.0409,-0.0409,...,0.038100,0.038100,0.077167,0.0424,-0.0609,-0.060900,-0.0609,-0.059522,-0.0562,-0.182834
TCGA-24-0979-01A-01D-0428-01.bed,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.522517,0.5523,0.5523,-1.5957,...,-0.100367,0.367999,0.304042,0.1481,0.4255,0.346981,0.3877,0.465632,0.4889,-0.310900
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.005300,0.0053,0.0053,0.0053,...,0.027900,0.020121,0.017100,0.0171,0.0276,0.027600,0.0276,0.027600,0.0276,0.006300
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.021400,-0.0214,-0.0214,-0.0214,...,-0.006000,-0.006000,-0.006000,-0.0060,-0.5160,-0.516000,-0.5160,-0.199739,-0.0135,-0.476522


In [109]:
pd.read_pickle("../data/cna/tcga_cna_raw_all_samples_all_chr_0.9_threshold.pkl").head(5)

,chr1_0:814,chr1_815:1095,chr1_1096:1509,chr1_1510:1511,chr1_1512:2235,chr1_2236:4371,chr1_4372:4373,chr1_4374:5346,chr1_5347:5351,chr1_5352:5609,...,chr22_12263:12548,chr22_12549:12550,chr22_12551:13709,chr22_13710:13715,chr22_13716:13870,chr22_13871:14465,chr22_14466:14506,chr22_14507:14508,chr22_14509:14513,chr22_14514:14530
TCGA-AA-3693-01A-01D-0903-01.bed,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,-0.0066,...,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457,0.0457
TCGA-4P-AA8J-01A-11D-A390-01.bed,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,-0.0409,...,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245,-0.2245
TCGA-24-0979-01A-01D-0428-01.bed,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,0.2214,...,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109,-0.3109
TCGA-4W-AA9T-01A-11D-A390-01.bed,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,0.0053,...,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063,0.0063
TCGA-5P-A9KC-01A-11D-A42I-01.bed,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,-0.0214,...,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776,-0.4776


In [132]:
[i for i in range(1,23) + "X"]

TypeError: unsupported operand type(s) for +: 'range' and 'str'